In [1]:
# 環境構築

!apt-get update
!apt-get install -y libsndfile1-dev
!pip install scipy
!pip install scikit-learn
!pip install librosa

/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/a

In [2]:
# clips 作成

SAMPLING_RATE = 44100 # 変更不可

clip_size = 81920 # 楽曲を再構築するパーツ1つあたりの大きさ
step_size = 20480 # clip をずらすときの大きさ
window_size = 10240 # CQT_CHROMA を取得するのに使用するサンプル数
hop_size = 640 # window をずらすときの大きさ

from scipy.io.wavfile import read, write
import glob
import os
import numpy as np
import librosa
import sys

if os.path.isfile("../data/out/clips4predict.wav"):
    print("../data/out/clips4predict.wav exists.")
    files = ["../data/out/clips4predict.wav"]
else:
    # clips.npy をもとにデータを作成する 
    if os.path.isfile("../data/arrays/clips4predict.npy"):
        print("loading ../data/arrays/clips4predict.npy ...")
        clips = np.load("../data/arrays/clips4predict.npy")
        print("creating ../data/out/clips4predict.wav ...")
        write("../data/out/clips4predict.wav", SAMPLING_RATE, clips.reshape((clips.shape[0] * clips.shape[1], )))
        files = ["../data/out/clips4predict.wav"]
        
    # clips,npy がないとき /data/wav44100 内の WAV ファイルを参照する
    else:
        files = glob.glob("../data/wav4predict/*.wav")
        #files = ["../data/wav44100/3DEmbodimentFromLines.wav"] # デバッグ用 
        
if len(files) == 0:
    print("WAV ファイルが見つかりませんでした。")
    sys.exit(1)
        
clips_filename = "../data/arrays/c" + str(clip_size) + "_s" + str(step_size) + "_f32_clips4predict"

if os.path.isfile(clips_filename + ".npy"):
    print("loading " + clips_filename + ".npy ...")
    clips = np.load(clips_filename + ".npy")
else:
    print("creating " + clips_filename + ".npy ...")
    raw_data_list = [librosa.load(file, sr=SAMPLING_RATE)[0] for file in files] # 左の音だけ使う

    clips = np.zeros((0, clip_size), dtype=np.float32)
    for raw_data in raw_data_list:
        tmp = [raw_data[i:i + clip_size] for i in range(0, len(raw_data) - clip_size, step_size)]
        clips = np.vstack((clips, np.array(tmp)))
    np.save(clips_filename, clips)

print("The clip array has " + str(clips.shape[0]) + " clips.")

loading ../data/arrays/c81920_s20480_f32_clips4predict.npy ...
The clip array has 41637 clips.


In [3]:
# cqt 作成

n_bins = 84

cqt_filename = clips_filename + "_w" + str(window_size) + "_h" + str(hop_size) + "_cqts4predict"

if os.path.isfile(cqt_filename + ".npy"):
    print("loading " + cqt_filename + ".npy ...")
    cqts = np.load(cqt_filename + ".npy")
else:
    print("creating " + cqt_filename + ".npy ...")
    num = 0
    for clip in clips:
        print("cqt progress: clip " + str(num + 1) + " / " + str(len(clips)))
        
        tmp_cqt = librosa.cqt(clip, sr=SAMPLING_RATE, hop_length=hop_size, n_bins=n_bins)
        tmp_cqt = tmp_cqt.reshape((1, tmp_cqt.shape[0], tmp_cqt.shape[1]))
        
        if num == 0:
            cqts = tmp_cqt
        else:
            cqts = np.vstack((cqts, tmp_cqt))
            
        num += 1

    np.save(cqt_filename, cqts)
    
print("The cqt array has " + str(cqts.shape[0]) + " cqts.")
print("cqts.shape: " + str(cqts.shape))
print("Type(cqts[0][0][0]): " + str(type(cqts[0][0][0])))
print("np.max(cqts[0][0]): " + str(np.max(cqts[0][0])))

loading ../data/arrays/c81920_s20480_f32_clips4predict_w10240_h640_cqts4predict.npy ...
The cqt array has 41637 cqts.
cqts.shape: (41637, 84, 129)
Type(cqts[0][0][0]): <class 'numpy.complex64'>
np.max(cqts[0][0]): (0.04157509+0.022495579j)


In [4]:
# cqt_choroma 作成

def Preprocessing(array):
    array = np.abs(array)
    array = np.log(array + 1)
    array = array / np.log(np.finfo(np.float32).max)
    array = array.T
    return array

cqt_chroma_filename = clips_filename + "_w" + str(window_size) + "_h" + str(hop_size) + "_cqt_chromas4predict"

if os.path.isfile(cqt_chroma_filename + ".npy"):
    print("loading " + cqt_chroma_filename + ".npy ...")
    cqt_chromas= np.load(cqt_chroma_filename + ".npy")
else:
    print("creating " + cqt_chroma_filename + ".npy ...")
    num = 0
    for cqt in cqts:
        print("cqt_chroma progress: clip " + str(num + 1) + " / " + str(len(cqts)))
        
        tmp_cqt_chroma = librosa.feature.chroma_cqt(C=cqt, sr=SAMPLING_RATE)
        tmp_cqt_chroma = Preprocessing(tmp_cqt_chroma)
        tmp_cqt_chroma = tmp_cqt_chroma.reshape((1, tmp_cqt_chroma.shape[0], tmp_cqt_chroma.shape[1]))
        
        if num == 0:
            cqt_chromas = tmp_cqt_chroma
        else:
            cqt_chromas = np.vstack((cqt_chromas, tmp_cqt_chroma))
            
        num += 1

    np.save(cqt_chroma_filename, cqt_chromas)
    
print("The cqt_chroma array has " + str(cqt_chromas.shape[0]) + " cqt_chromas.")
print("cqt_chromas.shape: " + str(cqt_chromas.shape)) # clip 番号、window 番号、 chroma 番号 になる
print("Type(cqt_chromas[0][0][0]): " + str(type(cqt_chromas[0][0][0])))
print("np.max(cqt_chromas[0][0]): " + str(np.max(cqt_chromas[0][0])))

loading ../data/arrays/c81920_s20480_f32_clips4predict_w10240_h640_cqt_chromas4predict.npy ...
The cqt_chroma array has 41637 cqt_chromas.
cqt_chromas.shape: (41637, 129, 12)
Type(cqt_chromas[0][0][0]): <class 'numpy.float32'>
np.max(cqt_chromas[0][0]): 0.0078125


In [5]:
# max_norm 作成

num_divide = 8

max_norm_filename = clips_filename + "_d" + str(num_divide) + "_max_norms4predict"

if os.path.isfile(max_norm_filename + ".npy"):
    print("loading " + max_norm_filename + ".npy ...")
    max_norms= np.load(max_norm_filename + ".npy")
else:
    print("creating " + max_norm_filename + ".npy ...")
    num = 0
    for clip in clips:
        print("max_norm progress: clip " + str(num + 1) + " / " + str(len(clips)))
        max_norm = [np.max(clip[int((i / num_divide) * len(clip)):int(((i + 1) / num_divide) * len(clip))] ** 2) for i in range(num_divide)]
    
        if num == 0:
            max_norms = np.array(max_norm)
        else:
            max_norms = np.vstack((max_norms, np.array(max_norm)))
        num += 1
        
    np.save(max_norm_filename, max_norms)
    
print("The max_norm array has " + str(max_norms.shape[0]) + " max_norms.")
print("max_norms.shape: " + str(max_norms.shape))
print("Type(max_norms[0][0]): " + str(type(max_norms[0][0])))
print("np.max(max_norms[0]): " + str(np.max(max_norms[0])))

loading ../data/arrays/c81920_s20480_f32_clips4predict_d8_max_norms4predict.npy ...
The max_norm array has 41637 max_norms.
max_norms.shape: (41637, 8)
Type(max_norms[0][0]): <class 'numpy.float32'>
np.max(max_norms[0]): 0.14306161


In [6]:
# 学習データとテストデータに分ける

def normalize(array):
    if (np.sum(array) == 0):
        return array
    else:
        return array / np.sum(array)
    
cqt_chroma_sum_threshold = 0.01 # 次の window の sum が閾値に満たないときに除外します
test_data_rate = 0.1

window_num_per_clip = cqt_chromas.shape[1]

cqt_chromas = np.array([np.hstack((cqt_chromas[i], np.repeat(np.array([max_norms[i]]), cqt_chromas.shape[1], axis=0))) for i in range(len(max_norms))])
cqt_chromas = np.concatenate([cqt_chromas[:-1, :, :], cqt_chromas[1:, 0, :].reshape(cqt_chromas.shape[0] - 1, 1, cqt_chromas.shape[2])], 1)

p = np.random.permutation(len(cqt_chromas))
cqt_chromas = cqt_chromas[p]
clips = clips[p]

x = np.delete(cqt_chromas, np.where(np.sum(cqt_chromas[:,-1,:], axis=1) < cqt_chroma_sum_threshold)[0], axis=0)

x_test = x[:int(x.shape[0] * test_data_rate), :window_num_per_clip, :].reshape(int(x.shape[0] * test_data_rate), window_num_per_clip, x.shape[2], 1)
x_train = x[int(x.shape[0] * test_data_rate):, :window_num_per_clip, :].reshape(x.shape[0] - int(x.shape[0] * test_data_rate), window_num_per_clip, x.shape[2], 1)

y = np.array([x[:, window_num_per_clip, i] for i in range(x.shape[2])])
y_maxs = np.max(y, axis=1)

y_tests =  np.array([x[:int(x.shape[0] * test_data_rate), window_num_per_clip, i] / y_maxs[i] for i in range(x.shape[2])])
y_trains = np.array([x[int(x.shape[0] * test_data_rate):, window_num_per_clip, i] / y_maxs[i] for i in range(x.shape[2])])

print("x_train.shape: " + str(x_train.shape))
print("x_test.shape: " + str(x_test.shape))
print("y_trains.shape: " + str(y_trains.shape))
print("y_tests.shape: " + str(y_tests.shape))

x_train.shape: (37439, 129, 20, 1)
x_test.shape: (4159, 129, 20, 1)
y_trains.shape: (20, 37439)
y_tests.shape: (20, 4159)


In [10]:
# 楽曲の出力

import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from keras.models import load_model
import soundfile as sf

batch_size = 64
epochs = 100
data_name = "c" + str(clip_size) + "_s" + str(step_size) + "_w" + str(window_size) + "_h" + str(hop_size) + "_d" + str(num_divide)

def add_fade(x, ft_len):
    r = np.arange(0, ft_len)*np.pi/ft_len
    w_fo = (0.5+0.5*np.cos(r))**0.5
    w_fi = (0.5-0.5*np.cos(r))**0.5
    
    x[0:ft_len]        *= w_fi
    x[clip_size-ft_len::] *= w_fo
    return x

def gen_xfade(x_pre, x_next, ft_len):
    if x_pre is None:
        xfade = x_next
    else:
        x_pre_begin = x_pre[:x_pre.shape[0] - ft_len]
        x_pre_end = x_pre[x_pre.shape[0] - ft_len:]
        x_pre_len = clip_size
        x_next_len = clip_size
        x_pre_len -= ft_len
        x_next_len -= ft_len
        xfade = np.concatenate((x_pre_begin, np.concatenate((x_pre_end, np.zeros(x_next_len))) + x_next))
    return xfade

def create_music_consider_weights(fname, rw):
    first_index = np.random.randint(0, len(cqt_chromas))
    predict_index = first_index
    out = None
    fade_samples = 10240
    
    for i in range(num_clips):
        print("-- generate " + str(i + 1) + " / " + str(num_clips))
        predict_vec = np.zeros((0, ))
        for i in range(x.shape[2]):
            predict = models[i].predict(np.array([cqt_chromas[predict_index, :-1]]))[0] * y_maxs[i]
            predict_vec = np.hstack((predict_vec, predict))
        predict_chroma = predict_vec[:len(predict_vec)-num_divide]
        predict_rhythm = predict_vec[len(predict_vec)-num_divide:]

        index = 0
        similar_index = 0
        score = -1 - rw
        chroma_score = 0
        rhythm_score = 0
        for cqt_chroma in cqt_chromas[:, (int)(fade_samples / hop_size)]:
            cqt_chroma_chroma = cqt_chroma[:len(cqt_chroma)-num_divide]
            cqt_chroma_rhythm = cqt_chroma[len(cqt_chroma)-num_divide:]
            tmp_chroma_score = cosine_similarity(np.array([predict_chroma]), np.array([cqt_chroma_chroma]))
            tmp_rhythm_score = cosine_similarity(np.array([predict_rhythm]), np.array([cqt_chroma_rhythm]))
            tmp_score = tmp_chroma_score + rw * tmp_rhythm_score
            if tmp_score > score:
                score = tmp_score
                similar_index = index
            index += 1

        print("score: " + str(score))
        print("predict_vec: " + str(predict_vec))
        print("cqt_chromas[similar_index]: " + str(cqt_chromas[similar_index, 0]))
        print("similar_index: " + str(similar_index))
        print("--")

        predict_index = similar_index

        tmp = add_fade(clips[predict_index], fade_samples)
        out = gen_xfade(out, tmp, fade_samples)
        
    sf.write(fname, out, SAMPLING_RATE, subtype="PCM_16")

models = []
for i in range(x.shape[2]):
    model_name = "independent_" + data_name + "_batch" + str(batch_size) + "_e" + str(epochs) + "_" + str(i)
    
    models.append(load_model("../data/models/" + model_name))

num_clips = 30
rhythm_weight = 0.4

for i in range(5):
    file_name = "out_" + "independent_" + data_name + "_batch" + str(batch_size) + "_e" + str(epochs) + "_rw" + format(rhythm_weight, '.2f').replace('.', '_') + "_PREDICT"
    fname = "../data/out/" + file_name + "_track" + str(i) + ".wav"
    print("creating " + fname + " ...")
    create_music_consider_weights(fname, rhythm_weight)

creating ../data/out/out_independent_c81920_s20480_w10240_h640_d8_batch64_e100_rw0_40_PREDICT_track0.wav ...
-- generate 1 / 30
1/1 [==============================] - 0s 36ms/step
score: [[1.33075179]]
predict_vec: [0.00267956 0.00209739 0.00186941 0.00369274 0.00047632 0.00053961
 0.00234138 0.00262477 0.00210223 0.00266891 0.00343175 0.00669725
 0.00727589 0.00766146 0.04091993 0.02545399 0.04270293 0.03521037
 0.04723246 0.05045326]
cqt_chromas[similar_index]: [0.         0.         0.         0.00356459 0.         0.
 0.0078125  0.         0.         0.         0.00388939 0.
 0.05528267 0.06283614 0.09939965 0.08483225 0.10187801 0.13191783
 0.14120926 0.1529696 ]
similar_index: 23586
--
-- generate 2 / 30
1/1 [==============================] - 0s 16ms/step
score: [[1.31648282]]
predict_vec: [0.00306146 0.00128466 0.00157156 0.00226353 0.00428203 0.00400133
 0.00441437 0.00142898 0.00052297 0.00078291 0.00330176 0.00213712
 0.05538129 0.04696257 0.05679985 0.07417177 0.0752061  0.0

1/1 [==============================] - 0s 17ms/step
score: [[1.34177063]]
predict_vec: [0.00199462 0.00017658 0.00531137 0.0019101  0.00452049 0.00216919
 0.00032179 0.00222688 0.00600664 0.00258788 0.00162198 0.00188347
 0.05215992 0.05781556 0.05376103 0.03478886 0.05063157 0.04292328
 0.06394342 0.05761481]
cqt_chromas[similar_index]: [0.0078125  0.         0.         0.00187997 0.         0.
 0.         0.         0.         0.         0.         0.
 0.11956341 0.11503789 0.10232657 0.06660111 0.07497683 0.10832793
 0.12602197 0.11438672]
similar_index: 30133
--
-- generate 7 / 30
1/1 [==============================] - 0s 16ms/step
score: [[1.32676898]]
predict_vec: [0.00609127 0.00042297 0.00565651 0.002506   0.00029435 0.00035189
 0.00054032 0.00118458 0.00234078 0.00094373 0.00334596 0.00472899
 0.05718338 0.03685555 0.04003307 0.0620863  0.06621174 0.06329397
 0.06869106 0.06945729]
cqt_chromas[similar_index]: [0.         0.         0.         0.         0.00560543 0.
 0.001875

1/1 [==============================] - 0s 15ms/step
score: [[1.34076483]]
predict_vec: [0.0006738  0.00104379 0.00019057 0.00101141 0.00089549 0.00594712
 0.00332543 0.00099002 0.00158799 0.00226905 0.00605003 0.0012873
 0.04317776 0.02961007 0.05210651 0.0347233  0.04221478 0.03608465
 0.04808182 0.05127425]
cqt_chromas[similar_index]: [0.00499189 0.00233071 0.00262728 0.         0.         0.
 0.         0.         0.         0.         0.00554133 0.0078125
 0.04968458 0.05951527 0.05528985 0.06451488 0.03244682 0.04734555
 0.0629586  0.06503528]
similar_index: 1674
--
-- generate 12 / 30
1/1 [==============================] - 0s 17ms/step
score: [[1.33569339]]
predict_vec: [0.00619703 0.00696091 0.00152477 0.00116625 0.00130556 0.00134258
 0.00254778 0.00379723 0.00048025 0.00088782 0.00458939 0.00711297
 0.03186449 0.0357348  0.01706857 0.028016   0.03353338 0.03604417
 0.05295379 0.04677682]
cqt_chromas[similar_index]: [0.         0.         0.0078125  0.00726355 0.         0.
 0.

1/1 [==============================] - 0s 15ms/step
score: [[1.32004006]]
predict_vec: [0.00025429 0.00044466 0.00353274 0.00188406 0.00271348 0.00579848
 0.00400475 0.00103288 0.00187168 0.00121738 0.00067776 0.00107652
 0.00687586 0.00515492 0.02165181 0.01072832 0.0059634  0.02435448
 0.03455274 0.03358193]
cqt_chromas[similar_index]: [0.         0.0078125  0.00405605 0.00441086 0.00384953 0.00533136
 0.         0.00291623 0.00092973 0.00405244 0.00147477 0.00539434
 0.04201949 0.02993044 0.08660131 0.05784459 0.09878483 0.14382446
 0.18866573 0.1681906 ]
similar_index: 36250
--
-- generate 17 / 30
1/1 [==============================] - 0s 22ms/step
score: [[1.30659586]]
predict_vec: [0.00130216 0.0020362  0.00580807 0.0019111  0.00164927 0.00058607
 0.00378424 0.00527376 0.00130397 0.00103067 0.00102061 0.00309142
 0.04713438 0.03167187 0.05441489 0.08025952 0.10043371 0.09402431
 0.09024211 0.08663387]
cqt_chromas[similar_index]: [0.         0.         0.         0.         0.    

1/1 [==============================] - 0s 16ms/step
score: [[1.32912172]]
predict_vec: [0.00613776 0.00706659 0.00188691 0.00227122 0.00487499 0.00086076
 0.00093623 0.00403481 0.00387022 0.0018407  0.00011512 0.00689765
 0.01523438 0.03327444 0.05784534 0.06388607 0.04408837 0.04161382
 0.05456932 0.05644171]
cqt_chromas[similar_index]: [0.         0.0078125  0.00446612 0.         0.         0.0028672
 0.         0.         0.00260341 0.00158234 0.         0.00469497
 0.07534496 0.06418973 0.08709595 0.07872435 0.06812979 0.09658157
 0.09696131 0.10689635]
similar_index: 23978
--
-- generate 22 / 30
1/1 [==============================] - 0s 15ms/step
score: [[1.32765992]]
predict_vec: [0.00046033 0.00124415 0.00153128 0.0005538  0.0012257  0.00050611
 0.00033116 0.0038192  0.00242987 0.00318329 0.00295585 0.005814
 0.04762559 0.04422304 0.03689572 0.05747148 0.05054978 0.05993897
 0.0661803  0.06181169]
cqt_chromas[similar_index]: [0.0078125  0.00413082 0.00434247 0.00430088 0.       

...

1/1 [==============================] - 0s 16ms/step
score: [[1.33072647]]
predict_vec: [0.00110859 0.00172563 0.00069904 0.00193474 0.00362153 0.00493653
 0.0009289  0.00179658 0.00135729 0.00092721 0.00051101 0.00278147
 0.04962649 0.02796204 0.02040968 0.03013483 0.04567858 0.0360937
 0.05179296 0.0513079 ]
cqt_chromas[similar_index]: [0.00428636 0.0017916  0.00730019 0.         0.         0.
 0.00773114 0.00609566 0.0078125  0.00557653 0.00120563 0.00451106
 0.1105388  0.14512362 0.1117495  0.18825504 0.10081904 0.05012772
 0.1476808  0.24660887]
similar_index: 40022
--
-- generate 6 / 30
1/1 [==============================] - 0s 17ms/step
score: [[1.33691092]]
predict_vec: [0.00508174 0.00214548 0.00052188 0.00017378 0.00423957 0.00439422
 0.00605489 0.00259406 0.0005924  0.00047001 0.00030378 0.00384131
 0.06250147 0.10529964 0.05616939 0.02962133 0.07784287 0.13581643
 0.12119672 0.10870036]
cqt_chromas[similar_index]: [0.0078125  0.00536217 0.00417004 0.00108219 0.00201363 0.
 0

1/1 [==============================] - 0s 16ms/step
score: [[1.30916251]]
predict_vec: [0.00195153 0.00194706 0.00143364 0.00147343 0.00174607 0.00412469
 0.00560575 0.00065792 0.00205728 0.00095436 0.00040813 0.00504849
 0.0660795  0.09304101 0.08543704 0.05952111 0.01788356 0.08982122
 0.08166654 0.07598177]
cqt_chromas[similar_index]: [0.00100048 0.00223761 0.00408531 0.         0.         0.
 0.00513558 0.00417956 0.00302151 0.         0.00628471 0.0078125
 0.05592313 0.05027815 0.08860651 0.03627509 0.01305833 0.12073758
 0.15571511 0.10773613]
similar_index: 847
--
-- generate 11 / 30
1/1 [==============================] - 0s 16ms/step
score: [[1.30831876]]
predict_vec: [0.00095584 0.00310851 0.00142232 0.00101248 0.00049831 0.00208457
 0.00042108 0.00340414 0.00241743 0.00261483 0.00302959 0.00508141
 0.0485359  0.01935922 0.00712192 0.06833037 0.08244464 0.06053539
 0.06481363 0.06813915]
cqt_chromas[similar_index]: [0.00241151 0.0078125  0.         0.00102171 0.         0.0038

1/1 [==============================] - 0s 16ms/step
score: [[1.35181323]]
predict_vec: [0.00040718 0.0002023  0.00628568 0.00555381 0.00057433 0.00497086
 0.00082982 0.001175   0.00095277 0.0015952  0.00432797 0.00112665
 0.01585506 0.01577807 0.01505202 0.03078718 0.03140968 0.03001616
 0.04130814 0.04535459]
cqt_chromas[similar_index]: [0.00349123 0.00703145 0.         0.         0.         0.00303049
 0.00508051 0.         0.         0.00383259 0.0078125  0.
 0.08424662 0.06104374 0.05228217 0.06119463 0.10880052 0.14345434
 0.1766549  0.20708221]
similar_index: 26451
--
-- generate 16 / 30
1/1 [==============================] - 0s 17ms/step
score: [[1.3124435]]
predict_vec: [0.0052498  0.00131675 0.00083097 0.00276156 0.00452814 0.00032349
 0.00307359 0.0016827  0.0034795  0.00062644 0.0004975  0.00146669
 0.03048673 0.03364052 0.06203815 0.08032668 0.09293158 0.11436802
 0.10021204 0.0981482 ]
cqt_chromas[similar_index]: [0.         0.         0.0078125  0.         0.0071088  0.00

1/1 [==============================] - 0s 16ms/step
score: [[1.34413035]]
predict_vec: [0.00546169 0.00043967 0.00202222 0.00081544 0.00127503 0.00090531
 0.00220561 0.00098431 0.00053861 0.00363486 0.00407775 0.00685679
 0.04902789 0.06314049 0.07405827 0.0604092  0.10846662 0.07867476
 0.08673387 0.08316553]
cqt_chromas[similar_index]: [0.         0.         0.00655766 0.00594397 0.         0.
 0.0072443  0.00476576 0.00657615 0.0078125  0.00596329 0.
 0.16638073 0.20127688 0.20068859 0.2991557  0.26799768 0.29780522
 0.1596901  0.22064304]
similar_index: 33726
--
-- generate 21 / 30
1/1 [==============================] - 0s 18ms/step
score: [[1.32197511]]
predict_vec: [0.00386579 0.00140469 0.00654782 0.00029871 0.00149144 0.00480643
 0.00052195 0.00042938 0.00144224 0.00298104 0.00514472 0.00266288
 0.11181248 0.16997768 0.14888781 0.16748604 0.08388808 0.12316025
 0.13496746 0.13420865]
cqt_chromas[similar_index]: [0.         0.         0.00632656 0.00097217 0.         0.00179503


1/1 [==============================] - 0s 18ms/step
score: [[1.34443667]]
predict_vec: [0.00299538 0.00215715 0.00081445 0.00016496 0.00178955 0.00363007
 0.00473083 0.00372022 0.00041531 0.00016286 0.00028935 0.00650106
 0.04325725 0.03181787 0.03197433 0.02875804 0.02404831 0.01776982
 0.03294773 0.0326289 ]
cqt_chromas[similar_index]: [0.0072705  0.         0.         0.         0.00348982 0.
 0.00497526 0.0078125  0.         0.         0.00387113 0.00288537
 0.14716524 0.169231   0.20979911 0.13178487 0.17850676 0.09003846
 0.12323168 0.18561614]
similar_index: 31358
--
-- generate 27 / 30
1/1 [==============================] - 0s 15ms/step
score: [[1.333671]]
predict_vec: [0.00414059 0.00248236 0.00122858 0.00033929 0.00118563 0.00055601
 0.00062022 0.00248753 0.00088017 0.00050217 0.00059191 0.00555309
 0.11569366 0.07399311 0.10011097 0.05267467 0.06429067 0.10212358
 0.09800033 0.09599683]
cqt_chromas[similar_index]: [0.004335   0.0021384  0.00606803 0.         0.         0.002